## Importing Packages

In [47]:
import numpy as np
import pandas as pd

## Defining Functions
- create_list_of_output_file_paths: takes in (1) the names of the folders containing model runs and (2) the name of the specific output file of interest, and returns a list of file paths 
- create_df_structure: takes in the list of file paths and creates a pandas dataframe structure to fill in with the output values from each scenario, w.r.t. dimensions of interest (e.g. tech, region, year, pollutant, etc)
- create_tableau_file: takes in (1) the dataframe structure, (2) list of file paths, and (3) list of dimensions of interest, and returns a filled in dataframe with a column for each model run

In [48]:
def create_list_of_output_file_paths(cases, file_name):
    list_of_output_file_paths = []
    for case in cases:
        list_of_output_file_paths.append("../" + case + "/outputs/" + file_name)
    
    return list_of_output_file_paths

In [49]:
def create_df_structure(list_of_output_file_paths):
    df_structure = pd.read_csv(list_of_output_file_paths[0])
    for i in range(1,len(list_of_output_file_paths)):
        df_structure = pd.concat([df_structure, pd.read_csv(list_of_output_file_paths[i])], axis=0)
    
    display(df_structure.head())
    col_titles = {}
    for col in df_structure.columns:
        col_title = input("What should the " + col + " column be called? If not a dimension of interest, label as drop.")
        col_titles[col] = col_title
    
    df_structure.rename(columns=col_titles, inplace=True)
    df_structure = df_structure.drop(columns=['drop']).drop_duplicates()
    
    return df_structure

In [50]:
def create_tableau_file(df_structure, list_of_output_file_paths):
    case_names = []
    for i in all_paths:
        case_names.append(i.split('/')[1])
        
    merge_cols = list(df_structure.columns)
    
    first_file = pd.read_csv(list_of_output_file_paths[0])
    display(first_file.head())
        
    col_titles = {}
    for col in first_file.columns:
        col_title = input("What should the " + col + " column be called? If not a dimension of interest, label as drop. If the column contains the metric of interest, label as Val.")
        col_titles[col] = col_title
    
    i=0
    for file_path in list_of_output_file_paths:
        file = pd.read_csv(file_path)
        file.rename(columns=col_titles, inplace=True)
        try:
            file = file.drop(columns=['drop'])
        except:
            pass
        file = file.groupby(merge_cols)['Val'].sum().reset_index()
        
        name_of_scenario = case_names[i]
        file.rename(columns={'Val': name_of_scenario}, inplace=True)
        
        df_structure = pd.merge(df_structure, file, on=merge_cols, how='left')
        i = i+1
        
    return df_structure

## Executing Functions

Step 1: Add name of folder containing new ReEDS run to the end of the list

In [51]:
scenarios = ['mid case outputs', 
             'mid 95by2050', 
             'IRA mid case',
             'IRA 95 by 2035',
             'IRA with NEEDS database',  
             'coal 2035 IRA NEEDS', 
             'coal CF IRA NEEDS',
             'IPM CCS coal 2035 IRA'] 

Step 2: Change the second argument to the output file of interest and run the cell below to generate a list of file paths that lead to the output file of interest for all model runs.
- Capacity: cap.csv
- Generation: gen_ann.csv
- Emissions: emit_irt.csv
- LCOE: lcoe_fullpol.csv
- System Cost: systemcost_ba_bulk.csv
- Health Damages: health_damages_caused_r.csv

In [53]:
all_paths = create_list_of_output_file_paths(scenarios, 'health_damages_caused_r.csv')

Step 3: Run the cell below and follow prompts. In this step, we are creating a dataframe structure that we will populate with actual values in Step 4. Only keep the columns that are the dimensions of interest (e.g. tech, region, year, pollutant, model, cr, etc). For the columns that are not needed, label them as 'drop' when prompted. Also make sure to drop the columns with the actual metric values (e.g. Val, md, damage_$, mortality). 

In [54]:
df_struct = create_df_structure(all_paths)

,reeds_ba,state_abbr,year,pollutant,tonnes,model,cr,md,damage_$,mortality
0,p1,WA,2010,SO2,47.03,AP2,ACS,3649.64,171657.09,0.0248
1,p1,WA,2010,SO2,47.03,AP2,H6C,9375.90,440986.30,0.0636
2,p1,WA,2010,SO2,47.03,EASIUR,ACS,18204.36,856224.00,0.1235
3,p1,WA,2010,SO2,47.03,EASIUR,H6C,46771.73,2199861.97,0.3173
4,p1,WA,2010,SO2,47.03,InMAP,ACS,4154.93,195423.01,0.0282


What should the reeds_ba column be called? If not a dimension of interest, label as drop.Region
What should the state_abbr column be called? If not a dimension of interest, label as drop.drop
What should the year column be called? If not a dimension of interest, label as drop.Year
What should the pollutant column be called? If not a dimension of interest, label as drop.Pollutant
What should the tonnes column be called? If not a dimension of interest, label as drop.drop
What should the model column be called? If not a dimension of interest, label as drop.Model
What should the cr column be called? If not a dimension of interest, label as drop.CR
What should the md column be called? If not a dimension of interest, label as drop.drop
What should the damage_$ column be called? If not a dimension of interest, label as drop.drop
What should the mortality column be called? If not a dimension of interest, label as drop.drop


Step 4: Run the cell below and follow prompts. Make sure that the column names are the same as in the previous step, otherwise the code will error. In this step, label the column with the metric of interest as 'Val'.

In [55]:
d = create_tableau_file(df_struct, all_paths)

,reeds_ba,state_abbr,year,pollutant,tonnes,model,cr,md,damage_$,mortality
0,p1,WA,2010,SO2,47.03,AP2,ACS,3649.64,171657.09,0.0248
1,p1,WA,2010,SO2,47.03,AP2,H6C,9375.90,440986.30,0.0636
2,p1,WA,2010,SO2,47.03,EASIUR,ACS,18204.36,856224.00,0.1235
3,p1,WA,2010,SO2,47.03,EASIUR,H6C,46771.73,2199861.97,0.3173
4,p1,WA,2010,SO2,47.03,InMAP,ACS,4154.93,195423.01,0.0282


What should the reeds_ba column be called? If not a dimension of interest, label as drop. If the column contains the metric of interest, label as Val.Region
What should the state_abbr column be called? If not a dimension of interest, label as drop. If the column contains the metric of interest, label as Val.drop
What should the year column be called? If not a dimension of interest, label as drop. If the column contains the metric of interest, label as Val.Year
What should the pollutant column be called? If not a dimension of interest, label as drop. If the column contains the metric of interest, label as Val.Pollutant
What should the tonnes column be called? If not a dimension of interest, label as drop. If the column contains the metric of interest, label as Val.drop
What should the model column be called? If not a dimension of interest, label as drop. If the column contains the metric of interest, label as Val.Model
What should the cr column be called? If not a dimension of interest,

In [56]:
d

,Region,Year,Pollutant,Model,CR,mid case outputs,mid 95by2050,IRA mid case,IRA 95 by 2035,IRA with NEEDS database,coal 2035 IRA NEEDS,coal CF IRA NEEDS,IPM CCS coal 2035 IRA
0,p1,2010,SO2,AP2,ACS,3649.64,3649.64,3649.64,3649.64,3649.64,3649.64,3649.64,3649.64
1,p1,2010,SO2,AP2,H6C,9375.90,9375.90,9375.90,9375.90,9375.90,9375.90,9375.90,9375.90
2,p1,2010,SO2,EASIUR,ACS,18204.36,18204.36,18204.36,18204.36,18204.36,18204.36,18204.36,18204.36
3,p1,2010,SO2,EASIUR,H6C,46771.73,46771.73,46771.73,46771.73,46771.73,46771.73,46771.73,46771.73
4,p1,2010,SO2,InMAP,ACS,4154.93,4154.93,4154.93,4154.93,4154.93,4154.93,4154.93,4154.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,p129,2030,NOX,AP2,H6C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.17
54908,p129,2030,NOX,EASIUR,ACS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6623.44
54909,p129,2030,NOX,EASIUR,H6C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17016.58
54910,p129,2030,NOX,InMAP,ACS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1594.27


Step 5: Export dataframe as csv file. Change the file name as appropriate. Following the naming conventions below will smooth the process of loading them into the Tableau viewer.
- <i>Capacity:</i> cap_all.csv
- <i>Generation:</i> gen_ann_all.csv
- <i>Emissions:</i> emit_irt_all.csv
- <i>LCOE:</i> lcoe_fullpol_all.csv
- <i>System cost:</i> systemcost_ba_bulk_all.csv
- <i>Average marginal damages:</i> average_marginal_damages_all.csv
- <i>Monetized annual health damages:</i> annual_health_damages_all.csv
- <i>Mortality rates:</i> mortality_all.csv

In [57]:
d.to_csv('average_marginal_damages_all.csv', index=False)